In [1]:
import torch 
import lightning as ln
import torch.nn as nn 
import torch.nn.functional as Func 
import pytorch_lightning as pl 
import torchmetrics
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader

import os
import pandas as pd
from pandas import DataFrame
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
train_path = 'cyberbullying_dataset.csv'
pd_dataset = pd.read_csv(train_path)

pd_dataset

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",3
1,Why is #aussietv so white? #MKR #theblock #ImA...,3
2,@XochitlSuckkks a classy whore? Or more red ve...,3
3,"@Jason_Gio meh. :P thanks for the heads up, b...",3
4,@RudhoeEnglish This is an ISIS account pretend...,3
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",1
47688,Turner did not withhold his disappointment. Tu...,1
47689,I swear to God. This dumb nigger bitch. I have...,1
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,1


In [3]:
pd_dataset.shape

(47692, 2)

In [4]:
# split_ratio = 0.2

# Calculate row indices for each split
# split_index = int(len(pd_dataset) * split_ratio)

# # Split the DataFrame into two parts
# df1 = pd_dataset.iloc[:split_index, :]  # First half
# df2 = pd_dataset.iloc[split_index:, :]  # Second half

# # Save as CSV files
# df1.to_csv('train.csv', index=False)
# df2.to_csv('valid.csv', index=False)

# print("DataFrames split and saved as CSV files!")

In [5]:
print(pd_dataset['cyberbullying_type'].value_counts())
print('#' * 20)
print(pd_dataset.info())

cyberbullying_type
5    7998
0    7992
2    7973
1    7961
3    7945
4    7823
Name: count, dtype: int64
####################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          47692 non-null  object
 1   cyberbullying_type  47692 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 745.3+ KB
None


In [6]:
le = LabelEncoder()

pd_dataset['cyberbullying_type'] = le.fit_transform(pd_dataset['cyberbullying_type'])
classes  = le.classes_

print(classes)
print('')
print(pd_dataset.columns)

[0 1 2 3 4 5]

Index(['tweet_text', 'cyberbullying_type'], dtype='object')


In [28]:
pd_dataset.describe()

,cyberbullying_type
count,47692.000000
mean,2.495681
std,1.709858
min,0.000000
25%,1.000000
50%,2.000000
75%,4.000000
max,5.000000


In [25]:
print(pd_dataset.head())

                                          tweet_text  cyberbullying_type
0  In other words #katandandre, your food was cra...                   3
1  Why is #aussietv so white? #MKR #theblock #ImA...                   3
2  @XochitlSuckkks a classy whore? Or more red ve...                   3
3  @Jason_Gio meh. :P  thanks for the heads up, b...                   3
4  @RudhoeEnglish This is an ISIS account pretend...                   3


In [7]:
tokenizer = get_tokenizer('basic_english')

In [8]:
# # Accessing the data

# # class TweetDataset(pl.LightningDataModule):
# #     def __init__(self, csv_file, batch_size=32, val_size=0.2, test_size=0.2, random_state=42, transform=None):
# #         super().__init__()
# #         self.csv_file = csv_file
# #         self.batch_size = batch_size
# #         self.tokenizer = get_tokenizer('basic_english')
# #         self.vocab = None
# #         self.input_size = None
# #         self.val_size = val_size
# #         self.test_size = test_size
# #         self.random_state = random_state
# #         self.transform = None

#     def setup(self, stage=None):
#         data = pd.read_csv(self.csv_file)

#         self.train_data, self.test_data = train_test_split(data, test_size=self.val_size + self.test_size, random_state=self.random_state)
#         self.val_data, self.test_data = train_test_split(self.test_data, test_size=self.test_size, random_state=self.random_state)

#         # self.train_data, self.val_data, self.test_data = train_test_split(data, test_size=self.val_size + self.test_size, random_state=self.random_state)
#         # print(len(self.train_data), len(self.val_data), len(self.test_data))

#         text_iterator = (text for text in self.train_data['tweet_text'])
#         self.vocab = build_vocab_from_iterator(text_iterator, specials=['<UNK>'])
#         self.vocab.set_default_index(self.vocab['<UNK>'])

#         self.input_size = len(self.vocab)
    

#     def __len__(self):
#         if self.train_data is not None:
#             return len(self.train_data)
#         elif self.val_data is not None:
#             return len(self.val_data)
#         elif self.test_data is not None:
#             return len(self.test_data)
#         else:
#             return 0


        
#     # def __len__(self):
#     #     return len(self.data)

#     def __getitem__(self, index):
#         if torch.is_tensor(index):
#             index = index.tolist()

#         if self.train_data is not None:
#             sample = self.train_data.iloc[index]
#         elif self.val_data is not None:
#             sample = self.val_data.iloc[index]
#         elif self.test_data is not None:
#             sample = self.test_data.iloc[index]
#         else:
#             raise ValueError("No data has been set up for the dataset.")

#         if self.transform:
#             sample = self.transform(sample)

#         return sample
    
#     def collate_fn(self, batch_idx):
#         text = [self.vocab(self.tokenizer(row['tweet_text'])) for row in batch_idx]
#         label = [row['cyberbullying_type'] for row in batch_idx]
       
#         return torch.tensor(text), torch.tensor(label)


#     def train_dataloader(self):
#         return DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True, collate_fn=self.collate_fn, num_workers=3)
    
#     def val_dataloader(self):
#         return DataLoader(self.val_data, batch_size=self.batch_size, shuffle=False, collate_fn=self.collate_fn, num_workers=3)

#     def test_dataloader(self):
#         return DataLoader(self.test_data, batch_size=self.batch_size, shuffle=False, collate_fn=self.collate_fn, num_workers=3)

#     # def collate_fn(self, batch):
#     #     text = [self.vocab(self.tokenizer(row['tweet_text'])) for row in batch]
#     #     label = [row['cyberbullying_type'] for row in batch]

#     #     return torch.tensor(text), torch.tensor(label)

    
# def my_collate_fn(batch):
#     try:
#         return [self.dataset[idx] for idx in batch]
#     except KeyError as e:
#         print(f"KeyError occurred in DataLoader worker process: {e}")
#         raise

In [18]:
class TweetDataset(pl.LightningDataModule):
    def __init__(self, csv_file, batch_size=32, random_state=42):
        super().__init__()
        self.csv_file = csv_file
        self.batch_size = batch_size
        # self.val_size = val_size
        # self.test_size = test_size
        self.random_state = random_state
        self.tokenizer = get_tokenizer("basic_english")
        self.vocab = None
        self.input_size = None
        self.train_data = None
        # self.val_data = None
        # self.test_data = None

    def __getitem__(self, index):
        return self.data[index]
        
    def collate_fn(self, batch):
        text = [self.vocab(self.tokenizer(row['tweet_text'])) for row in batch]
        label = [row['cyberbullying_type'] for row in batch]
        return torch.tensor(text), torch.tensor(label)


    def setup(self, stage=None):
        self.train_data = pd.read_csv(self.csv_file)
        # self.train_data, temp_data = train_test_split(data, test_size=self.val_size + self.test_size, random_state=self.random_state)
        # self.val_data, self.test_data = train_test_split(temp_data, test_size=self.test_size, random_state=self.random_state)
        self.train_data.reset_index(drop=True, inplace=True)
        
        text_iterator = (text for text in self.train_data['tweet_text'])
        self.vocab = build_vocab_from_iterator(text_iterator, specials=['<UNK>'])
        self.vocab.set_default_index(self.vocab['<UNK>'])
        self.input_size = len(self.vocab)

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True, collate_fn=self.collate_fn, num_workers=3)

    # def val_dataloader(self):
    #     return DataLoader(self.val_data, batch_size=self.batch_size, shuffle=False, collate_fn=self.collate_fn, num_workers=3)

    # def test_dataloader(self):
    #     return DataLoader(self.test_data, batch_size=self.batch_size, shuffle=False, collate_fn=self.collate_fn, num_workers=3)
   
    def __len__(self):
        if self.train_data is not None:
            return len(self.train_data)
        # elif self.val_data is not None:
        #     return len(self.val_data)
        # elif self.test_data is not None:
        #     return len(self.test_data)
        else:
            return 0

    

In [19]:
dataset = TweetDataset('cyberbullying_dataset.csv')

dataset.setup()

print(len(dataset))

47692


In [11]:
# for batch in dataset:
#     print(batch)

In [12]:
# val_dataloader = DataLoader(dataset, batch_size=32, collate_fn=my_collate_fn)

# train_dataloader = DataLoader(dataset, batch_size=32, collate_fn=my_collate_fn)
# train_dataset = dataset.train_dataloader()
# val_dataset = dataset.val_dataloader()

# train_dataset

In [13]:
# val_dataloader

In [20]:
class TextClassifier(pl.LightningModule):
    def __init__(self, input_size, num_classes=6, lr=0.001):
        super().__init__()
        self.save_hyperparameters()
        # self.embedding = None

        pretrained_embeddings = torch.randn(input_size, 64)

        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=False)
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()

        self.train_acc = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)
        # self.val_acc = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)
        # self.test_acc = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)
    
    def forward(self, x):
        x = self.embedding(x)
        x = x.mean(dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

    def training_step(self, batch, batch_id):
        x, y = batch
        logit = self(x)
        loss = Func.cross_entropy(logits, y)
        preds = logits.argmax(dim=1)
        self.train_acc(preds, y)
        self.log('train_loss', loss)
        self.log('train_acc', self.train_acc)
        return loss 

    # def validation_step(self, batch, batch_id):
    #     x, y = batch
    #     logits = self(x)
    #     loss = F.cross_entropy(logits, y)
    #     preds = logits.argmax(dim=1)
    #     self.val_acc(preds, y)
    #     self.log('val_loss', loss)
    #     self.log('val_acc', self.val_acc)

    # def test_step(self, batch, batch_idx):
    #     x, y = batch
    #     logits = self(x)
    #     loss = F.cross_entropy(logits, y)
    #     preds = logits.argmax(dim=1)
    #     self.test_acc(preds, y)
    #     self.log('test_loss', loss)
    #     self.log('test_acc', self.test_acc)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        

In [21]:
detection_model = TextClassifier(input_size=dataset.input_size, num_classes=6)

detection_model

TextClassifier(
  (embedding): Embedding(807, 64)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=6, bias=True)
  (relu): ReLU()
  (train_acc): MulticlassAccuracy()
)

In [22]:
trainer = pl.Trainer(max_epochs=10)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [23]:
try:
    trainer.fit(detection_model, dataset)
except KeyError as e:
    print(f"KeyError occurred: {e}")
    


  | Name      | Type               | Params
-------------------------------------------------
0 | embedding | Embedding          | 51.6 K
1 | fc1       | Linear             | 4.2 K 
2 | fc2       | Linear             | 390   
3 | relu      | ReLU               | 0     
4 | train_acc | MulticlassAccuracy | 0     
-------------------------------------------------
56.2 K    Trainable params
0         Non-trainable params
56.2 K    Total params
0.225     Total estimated model params size (MB)
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7efd14e02950>Exception ignored in: 
Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7efd14e02950>
Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __d

Training: |          | 0/? [00:00<?, ?it/s]


  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'    
if w.is_alive():

KeyError occurred: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3791, in get_loc
    return self._engine.get_loc(casted_key)
  File "index.pyx", line 152, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 181, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 17219

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/pytho

AssertionError
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    

: assert self._parent_pid == os.getpid(), 'can only test a child process'can only test a child process

AssertionError: can only test a child process
